In [1]:
import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA
from scipy.sparse.linalg import lsqr, lsmr

In [2]:
def OMP_3(Y, T_0, D, batch_size = 2, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))
    D_norm = np.linalg.norm(D, axis=1, keepdims=True)

    splits = np.arange(0, Y.shape[0], step=batch_size)
    Y_batches = np.split(Y, splits)[1:]
    
    for i, y in enumerate(Y_batches):
        if i == (len(Y_batches) - 1):
            batch_size = np.arange(splits[i], len(Y)).shape[0]
            if debug:
                print(f'batch_size = {batch_size}')
        I = np.empty((batch_size, T_0), dtype=np.int32)
        D_I = np.zeros((batch_size, T_0, D.shape[1]))
        r = y   # (batch_size, D.shape[1])
        gamma = 0
        Q = np.empty((batch_size, D.shape[1], T_0))
        R = np.empty((batch_size, T_0, T_0))
        
        for j in range(T_0):
            if debug:
                print(f'Batch {i}, Iteration {j}')
                print(f'r shape: {r.shape}')
            D_r = np.abs(r @ D.T)
            k = np.argmax(D_r, axis = 1)
            
            I[:, j] = k
            D_I[:, j] = D[k]

            if j == 0:
                D_k_norm = D_norm[k]
                R[:, 0, 0] = D_k_norm[:, 0]
                Q[:, :, 0] = D[k] / D_k_norm
                
                if debug:
                    print(D_k_norm)
                    print(np.sum(Q[:, :, 0] ** 2, axis = 1))
                    print(R[:, 0, 0])

                gamma = np.squeeze(y[:, np.newaxis] @ Q[:, :, :1], axis=1) 
                gamma /= D_k_norm
                
            else:
                if debug:
                    print(np.transpose(Q[:, :, :j], (0, 2, 1)).shape)
                    print(D[k][..., np.newaxis].shape)
                # dot = np.transpose(Q[:, :, :j], (0, 2, 1)) @ D[k][..., np.newaxis]
                dot = np.squeeze(D[k][:, np.newaxis] @ Q[:, :, :j], axis=1)
                
                # R[:, 0:j, j] = dot[0]
                R[:, 0:j, j] = dot
                
                # q_j = D[k] - (Q[:, :, :j] @ dot)[..., 0]
                q_j = D[k] - np.squeeze(Q[:, :, :j] @ dot[..., np.newaxis], axis = -1)
                
                # q_j_norm = np.linalg.norm(q_j)
                q_j_norm = np.linalg.norm(q_j, axis = 1)
                
                R[:, j, j] = q_j_norm
                # Q[:, :, j] = q_j / q_j_norm
                Q[:, :, j] = q_j / q_j_norm[:, np.newaxis]
                

                if debug:            
                    print(Q[:, :, :j+1].shape)
                    print(y.shape)
                Q_T_y = np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1))
                gamma = LA.solve_triangular(R[:, :j+1, :j+1], 
                                        Q_T_y, 
                                        overwrite_b = True,
                                        check_finite = False)
                if debug:
                    print(f'original gamma shape: {gamma.shape}')
                # gamma = np.transpose(gamma, (0, 2, 1))
                gamma = np.squeeze(gamma, axis=-1)
                

            if debug:
                print(gamma.shape)
                print(f'D_I shape: {D_I[:, :j+1].shape}')
                print(f'y shape: {y.shape}')
            # est = (np.transpose(D_I[:, :j+1], (0, 2, 1)) @ gamma[..., np.newaxis])[..., 0] 
            est = np.squeeze(gamma[:, np.newaxis] @ D_I[:, :j+1], axis=1)
            if debug:
                print(f'est shape: {est.shape}')
            r = y - est
            # if r.ndim > 2:
                # r = np.transpose(r, (0, 2, 1))
            if debug:
                print(r.shape)
            if debug:
                print(np.sum(r * r, axis = -1))
                # print(res)

        # X[i, I] = gamma
        if i == (len(Y_batches) - 1):
            if debug:
                # print(splits[i].dtype)
                # print(I.dtype)
                print(f'gamma: {gamma}')
                print(f'gamma shape: {gamma.shape}')
                print(f'k shape: {k.shape}')
                print(f'I shape: {I.shape}')
            # if T_0 == 1:
            #     X[np.arange(splits[i], len(Y)), I[:, 0]] = gamma[:, 0]
            # else:
            #     X[np.arange(splits[i], len(Y))[:, np.newaxis], I] = gamma[:, 0, :]
            X[np.arange(splits[i], len(Y))[:, np.newaxis], I] = gamma
            
        elif i == 0:
            X[np.arange(0, splits[1])[:, np.newaxis], I] = gamma
        else:
            X[np.arange(splits[i], splits[i+1])[:, np.newaxis], I] = gamma

        if debug:
            print()
    return X

In [3]:
def OMP_verif(code):
    print(code)
    I = np.nonzero(code)
    print(I)
    print(code[I])

In [4]:
X_orig, y_orig = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [5]:
Y, T_0, D, batch_size = y_orig.T, 2, X_orig.T.copy(), 2

In [6]:
X = np.zeros((Y.shape[0], D.shape[0]))
D_norm = np.linalg.norm(D, axis=1, keepdims=True)

splits = np.arange(0, Y.shape[0], step=batch_size)
Y_batches = np.split(Y, splits)[1:]

## i = 0

In [7]:
i = 0
y = Y_batches[0]

In [8]:
I = np.empty((batch_size, T_0), dtype=np.int32)
D_I = np.zeros((batch_size, T_0, D.shape[1]))
r = y   # (batch_size, D.shape[1])
gamma = 0
Q = np.empty((batch_size, D.shape[1], T_0))
R = np.empty((batch_size, T_0, T_0))

## j = 0

In [9]:
j = 0

In [10]:
D.shape

(20, 50)

In [11]:
r.T.shape

(50, 2)

In [12]:
%timeit np.abs(D @ r.T)

2.43 μs ± 72.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
%timeit np.abs(r @ D.T)

2.59 μs ± 338 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
%timeit D_r = np.abs( \
                    np.einsum('bf, nf -> bn', r, D) \
                        )

9.28 μs ± 306 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%timeit D_r = np.abs( \
                    np.einsum('bf, nf -> bn', r, D, optimize = 'optimal'), \
                        )

35.7 μs ± 742 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
np.abs(r @ D.T)

array([[1095.85066332, 5727.78487008, 2068.71353893, 4002.68576918,
        5288.15627246,  897.92560519, 1048.97688818, 3083.69629081,
        4997.88028526, 1702.09198004,   57.68561827, 5810.32760456,
          70.48761555,   41.80389475, 4305.12062116,  418.31351006,
        1391.1113978 , 5056.10114459, 3002.48098006, 1243.45720009],
       [2478.66183746, 5887.68237315, 2039.70984467,  365.45070694,
        5274.6530587 , 1811.47604192,  535.26404001, 2751.0898888 ,
        5473.53689474, 2869.01012155,  185.55740682, 4373.17989079,
         575.17875571, 1002.68833866, 1140.96899389, 1667.18822542,
        4471.45963555, 2976.1444635 , 3299.30338842, 1291.41922235]])

In [17]:
D_r = np.abs( \
                    np.einsum('bf, nf -> bn', r, D, optimize = 'optimal'), \
                        )
D_r

array([[1095.85066332, 5727.78487008, 2068.71353893, 4002.68576918,
        5288.15627246,  897.92560519, 1048.97688818, 3083.69629081,
        4997.88028526, 1702.09198004,   57.68561827, 5810.32760456,
          70.48761555,   41.80389475, 4305.12062116,  418.31351006,
        1391.1113978 , 5056.10114459, 3002.48098006, 1243.45720009],
       [2478.66183746, 5887.68237315, 2039.70984467,  365.45070694,
        5274.6530587 , 1811.47604192,  535.26404001, 2751.0898888 ,
        5473.53689474, 2869.01012155,  185.55740682, 4373.17989079,
         575.17875571, 1002.68833866, 1140.96899389, 1667.18822542,
        4471.45963555, 2976.1444635 , 3299.30338842, 1291.41922235]])

In [18]:
D_r.shape

(2, 20)

In [19]:
k = np.argmax(D_r, axis=1)
k

array([11,  1])

In [20]:
I[:, 0] = k

In [21]:
I

array([[       11, 825425229],
       [        1,  65483093]], dtype=int32)

In [22]:
D_I[:, 0] = D[k]

In [23]:
D_I[0].shape

(2, 50)

## QR

In [24]:
D_k_norm = D_norm[k]

In [25]:
D_k_norm

array([[7.6339965 ],
       [6.33859759]])

In [26]:
R[:, 0, 0] = D_k_norm[:, 0]

In [27]:
R[:, 0, 0]

array([7.6339965 , 6.33859759])

In [28]:
Q[:, :, 0] = D[k] / D_k_norm

In [29]:
Q[:, :, 0]

array([[-0.09426313,  0.17375717,  0.13996724, -0.00146   , -0.08349978,
         0.03138366, -0.07841947,  0.10104877, -0.04637072, -0.01183941,
         0.18485364,  0.09149299,  0.05013526, -0.22243995, -0.06779136,
        -0.13735308,  0.26346052, -0.04836023, -0.05177222,  0.0493091 ,
        -0.01448005, -0.33442376,  0.08861161, -0.06523876, -0.19534429,
        -0.15573222, -0.0774696 , -0.18927703,  0.23107848,  0.02796439,
        -0.15262646, -0.15426113, -0.20309537,  0.24667953,  0.15119001,
        -0.12218252, -0.05911753, -0.02915978, -0.17011885,  0.04792363,
        -0.19147673, -0.25567265,  0.12175865,  0.19509519, -0.05067737,
         0.04709255, -0.17114585, -0.12522209, -0.04788463, -0.0880876 ],
       [ 0.04330869,  0.0094492 , -0.10851574, -0.13215511, -0.02096372,
         0.15343075,  0.12095155, -0.14336976, -0.10153956,  0.16319223,
        -0.0090666 ,  0.14701176,  0.17296363, -0.35584594, -0.06928812,
        -0.26918733,  0.0307749 ,  0.17348626, -0.

In [30]:
Q[:, :, 0].shape

(2, 50)

In [31]:
%timeit np.einsum('bn, bn -> b', Q[:, :, 0], Q[:, :, 0])

4.69 μs ± 70.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
%timeit np.sum(Q[:, :, 0] ** 2, axis = 1)

6.89 μs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [33]:
# gamma = (y[:, np.newaxis] @ Q[:, :, :j+1]) / D_k_norm

In [34]:
D_k_norm.shape

(2, 1)

In [35]:
y.shape

(2, 50)

In [36]:
Q[..., :1].shape

(2, 50, 1)

In [37]:
gamma = np.squeeze((y[:, np.newaxis] @ Q[:, :, :1]), axis=1)
gamma

array([[761.11216477],
       [928.86199072]])

In [38]:
np.squeeze((y[:, np.newaxis] @ Q[:, :, :1]), axis=1).shape

(2, 1)

In [39]:
%timeit np.squeeze((y[:, np.newaxis] @ Q[:, :, :1]), axis=1)

3.43 μs ± 61.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
%timeit np.einsum('bf, bfj -> bj', y, Q[..., :1])

4.45 μs ± 49.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [41]:
np.einsum('bf, bfj -> bj', y, Q[..., :1])

array([[761.11216477],
       [928.86199072]])

In [42]:
gamma /= D_k_norm
gamma

array([[ 99.70035543],
       [146.54061532]])

## Calc residual

In [43]:
D_I[:, :j+1].shape

(2, 1, 50)

In [44]:
gamma.shape

(2, 1)

In [45]:
est = np.squeeze(gamma[:, np.newaxis] @ D_I[:, :j+1], axis=1)
est

array([[ -71.74481331,  132.24869721,  106.5307669 ,   -1.11122146,
         -63.55269801,   23.88648631,  -59.68601029,   76.90944727,
         -35.29331877,   -9.01111825,  140.69435635,   69.63642602,
          38.15855931, -169.30175376,  -51.59683248, -104.54110331,
         200.52300601,  -36.80756046,  -39.40447005,   37.52975925,
         -11.02094282, -254.53399206,   67.44337545,  -49.65401235,
        -148.67891204, -118.52969056,  -58.96305619, -144.06105143,
         175.8766459 ,   21.28403675, -116.16585326, -117.41002613,
        -154.57835678,  187.75079383,  115.07255808,  -92.99460033,
         -44.99507321,  -22.19386551, -129.4795278 ,   36.47525419,
        -145.73526857, -194.59556266,   92.67198999,  148.48931879,
         -38.57116094,   35.84271642, -130.2611915 ,  -95.30805632,
         -36.4455736 ,  -67.04454566],
       [  40.22779598,    8.77700372, -100.79614676, -122.7538557 ,
         -19.4724016 ,  142.51599297,  112.34729438, -133.17072525,
         

In [46]:
est.shape

(2, 50)

In [47]:
np.einsum('bj, bjf -> bf', gamma, D_I[:, :j+1])

array([[ -71.74481331,  132.24869721,  106.5307669 ,   -1.11122146,
         -63.55269801,   23.88648631,  -59.68601029,   76.90944727,
         -35.29331877,   -9.01111825,  140.69435635,   69.63642602,
          38.15855931, -169.30175376,  -51.59683248, -104.54110331,
         200.52300601,  -36.80756046,  -39.40447005,   37.52975925,
         -11.02094282, -254.53399206,   67.44337545,  -49.65401235,
        -148.67891204, -118.52969056,  -58.96305619, -144.06105143,
         175.8766459 ,   21.28403675, -116.16585326, -117.41002613,
        -154.57835678,  187.75079383,  115.07255808,  -92.99460033,
         -44.99507321,  -22.19386551, -129.4795278 ,   36.47525419,
        -145.73526857, -194.59556266,   92.67198999,  148.48931879,
         -38.57116094,   35.84271642, -130.2611915 ,  -95.30805632,
         -36.4455736 ,  -67.04454566],
       [  40.22779598,    8.77700372, -100.79614676, -122.7538557 ,
         -19.4724016 ,  142.51599297,  112.34729438, -133.17072525,
         

In [48]:
%timeit np.squeeze(gamma[:, np.newaxis] @ D_I[:, :j+1], axis=1)

3.31 μs ± 22 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [49]:
%timeit np.einsum('bj, bjf -> bf', gamma, D_I[:, :j+1])

4.49 μs ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [50]:
est.shape

(2, 50)

In [51]:
y.shape

(2, 50)

In [52]:
r = y - est
r

array([[ -51.5460529 , -172.47630218, -186.99891326,  -47.25110175,
        -241.31153513,  213.51304266,   10.64433521,  -69.61105995,
        -405.18288147,  347.46572258,   32.5601317 ,  -42.89652637,
         -58.67172343, -344.75364291,  -48.63411688, -346.30292805,
         -29.27683511,  -31.17996064,  -80.79428742,   12.96085306,
         238.92616884,  202.56191481, -160.07530486,  286.04525558,
          36.6712503 ,   -5.82697629,  412.71400368,  248.08754194,
         264.02496446,   -6.43559703,  384.0748996 ,   69.17747929,
        -101.0001188 ,  132.65468401,  -44.12572542, -127.36535829,
          20.6272936 , -224.68291623,  577.69763392,  370.97602156,
         254.3653371 , -432.65111741,  -25.05826661,    7.67620306,
        -122.2865309 ,   34.47404397,  -40.69285074,  -43.46869773,
         264.9646971 , -398.17951738],
       [  -3.88961347, -163.11090503,   79.45632897,  301.67185408,
        -227.37650228, -144.88451238, -249.86669849,   58.29174689,
        -

In [53]:
np.sum(y ** 2, axis = 1)

array([3028716.79860624, 2680676.76775491])

In [54]:
np.sum(r ** 2, axis = 1)

array([2449425.07124399, 1817892.16995816])

## j = 1

In [55]:
j = 1

In [56]:
r.shape

(2, 50)

In [57]:
D.T.shape

(50, 20)

In [58]:
D_r = np.abs(r @ D.T)
D_r

array([[1.03355019e+03, 5.57380704e+03, 2.05237182e+03, 3.02583307e+03,
        5.19162413e+03, 7.46676418e+02, 3.36981937e+02, 8.73317896e+02,
        5.20702806e+03, 1.26423879e+03, 4.13010405e+02, 0.00000000e+00,
        7.30912395e+02, 1.80734299e+02, 4.19030906e+03, 7.32135694e+02,
        1.55201011e+03, 5.06067071e+03, 3.07686079e+03, 1.13477115e+03],
       [1.77213640e+03, 2.27373675e-12, 1.21905840e+03, 2.47865282e+02,
        3.87248846e+03, 1.75917115e+03, 1.47351152e+03, 2.93395661e+03,
        4.91822978e+03, 1.56292680e+03, 2.84731369e+02, 4.14686168e+03,
        8.76552565e+02, 1.27957306e+03, 5.43027893e+02, 1.36297776e+03,
        4.25225169e+03, 1.88943980e+03, 3.60219808e+03, 1.23049128e+03]])

In [59]:
k = np.argmax(D_r, axis=1)
k

array([1, 8])

In [60]:
I[:, j] = k

In [61]:
I

array([[11,  1],
       [ 1,  8]], dtype=int32)

In [62]:
D_I[:, j] = D[k]

In [63]:
D_I

array([[[-0.71960439,  1.32646164,  1.0685094 , -0.01114561,
         -0.63743703,  0.23958276, -0.59865394,  0.77140595,
         -0.35399391, -0.09038201,  1.41117206,  0.69845715,
          0.38273243, -1.69810582, -0.51751904, -1.04855297,
          2.01125668, -0.36918184, -0.39522898,  0.37642553,
         -0.11054066, -2.55298982,  0.67646073, -0.49803245,
         -1.49125759, -1.18885926, -0.59140267, -1.4449402 ,
          1.76405235,  0.21348005, -1.16514984, -1.17762896,
         -1.55042935,  1.8831507 ,  1.15418403, -0.93274091,
         -0.45130304, -0.22260568, -1.29868672,  0.36584879,
         -1.46173269, -1.9518041 ,  0.92950511,  1.48935596,
         -0.38687085,  0.3595044 , -1.30652685, -0.955945  ,
         -0.36555109, -0.67246045],
        [ 0.27451636,  0.05989468, -0.68783761, -0.83767804,
         -0.13288058,  0.97253579,  0.76666318, -0.90876325,
         -0.6436184 ,  1.03440989, -0.05746952,  0.93184837,
          1.09634685, -2.25556423, -0.43918952, -

## QR

In [64]:
# dot = np.transpose(Q[:, :, :j], (0, 2, 1)) @ D[k][..., np.newaxis]

In [65]:
Q[:, :, :j].shape

(2, 50, 1)

In [66]:
D[k].shape

(2, 50)

In [67]:
np.squeeze(D[k][:, np.newaxis] @ Q[:, :, :j], axis=1).shape

(2, 1)

In [68]:
dot = np.squeeze(D[k][:, np.newaxis] @ Q[:, :, :j], axis=1)
dot

array([[0.20230636],
       [0.59783598]])

In [69]:
R[:, 0:j, j].shape

(2, 1)

In [70]:
R[:, 0:j, j] = dot

In [71]:
Q[:, :, :j].shape

(2, 50, 1)

In [72]:
dot.shape

(2, 1)

In [73]:
D[k].shape

(2, 50)

In [74]:
np.squeeze(Q[:, :, :j] @ dot[..., np.newaxis], axis = -1).shape

(2, 50)

In [75]:
q_j = D[k] - np.squeeze(Q[:, :, :j] @ dot[..., np.newaxis], axis = -1)

In [76]:
q_j.shape

(2, 50)

In [77]:
np.sqrt(np.sum(q_j ** 2, axis = 1))

array([6.3353683 , 8.16693044])

In [78]:
q_j_norm = np.linalg.norm(q_j, axis = 1)
q_j_norm

array([6.3353683 , 8.16693044])

In [79]:
q_j_norm.shape

(2,)

In [80]:
R[:, j, j].shape

(2,)

In [81]:
R[:, j, j] = q_j_norm

In [82]:
R[:, j, j]

array([6.3353683 , 8.16693044])

In [83]:
Q[:, :, j].shape

(2, 50)

In [84]:
q_j.shape

(2, 50)

In [85]:
np.linalg.norm((q_j.T / q_j_norm).T, axis = 1)

array([1., 1.])

In [86]:
np.linalg.norm((q_j / q_j_norm[:, np.newaxis]), axis = 1)

array([1., 1.])

In [87]:
Q[:, :, j] = q_j / q_j_norm[:, np.newaxis]

In [88]:
np.linalg.norm(Q[:, :, j], axis=-1)

array([1., 1.])

In [89]:
Q[:, :, :j+1].shape

(2, 50, 2)

In [90]:
y.shape

(2, 50)

In [91]:
R[:, :j+1, :j+1].shape

(2, 2, 2)

In [92]:
(np.transpose(Q[:, :, :j+1], (0, 2, 1))).shape

(2, 2, 50)

In [93]:
(np.transpose(Q[:, :, :j+1], (0, 2, 1)) @ y[..., np.newaxis]).shape

(2, 2, 1)

In [94]:
np.transpose(Q[:, :, :j+1], (0, 2, 1)) @ y[..., np.newaxis]

array([[[761.11216477],
        [879.79210945]],

       [[928.86199072],
        [602.21276635]]])

In [95]:
np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1)).shape

(2, 2, 1)

In [96]:
np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1))

array([[[761.11216477],
        [879.79210945]],

       [[928.86199072],
        [602.21276635]]])

In [97]:
%timeit np.transpose(Q[:, :, :j+1], (0, 2, 1)) @ y[..., np.newaxis]

4.39 μs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [98]:
%timeit np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1))

4.39 μs ± 52.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Note: No squeeze here so solve_triangular works correctly !!!

In [99]:
Q_T_y = np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1))
Q_T_y

array([[[761.11216477],
        [879.79210945]],

       [[928.86199072],
        [602.21276635]]])

In [100]:
Q_T_y.shape

(2, 2, 1)

In [101]:
gamma = LA.solve_triangular(R[:, :j+1, :j+1], 
                            Q_T_y,
                            overwrite_b = False,
                            check_finite = False)

In [102]:
gamma.shape

(2, 2, 1)

In [103]:
gamma

array([[[ 96.02020336],
        [138.8699232 ]],

       [[139.58589021],
        [ 73.73795705]]])

In [104]:
np.squeeze(gamma, axis=-1)

array([[ 96.02020336, 138.8699232 ],
       [139.58589021,  73.73795705]])

In [105]:
gamma = np.squeeze(gamma, axis=-1)

In [106]:
gamma

array([[ 96.02020336, 138.8699232 ],
       [139.58589021,  73.73795705]])

In [107]:
LA.solve_triangular(R[0, :j+1, :j+1], 
                                        np.squeeze(np.transpose(y[:, np.newaxis] @ Q[:, :, :j+1], (0, 2, 1)), axis=-1)[0], 
                                        overwrite_b = True,
                                        check_finite = False)

array([ 96.02020336, 138.8699232 ])

# Calc res

In [108]:
gamma.shape

(2, 2)

In [109]:
D_I[:, :j+1].shape

(2, 2, 50)

In [110]:
gamma.ndim

2

In [111]:
# est = np.squeeze(np.transpose(gamma, (0, 2, 1)) @ D_I[:, :j+1], axis=1)

In [112]:
est = np.squeeze(gamma[:, np.newaxis] @ D_I[:, :j+1], axis=1)

In [113]:
est.shape

(2, 50)

In [114]:
np.sum(r**2, axis=1)

array([2449425.07124399, 1817892.16995816])

In [115]:
r = y - est

In [116]:
np.sum(r**2, axis=-1)

array([1675390.91540181, 1455231.95400867])

## Update X

In [117]:
splits[i]

np.int64(0)

In [118]:
len(Y)

2

In [119]:
I.shape

(2, 2)

In [120]:
gamma.shape

(2, 2)

In [121]:
X.ravel().shape

(40,)

In [122]:
X.shape

(2, 20)

In [123]:
index = np.arange(0, X.ravel().shape[0]).reshape(X.shape[0], X.shape[1])

In [124]:
index

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39]])

In [125]:
if i == (len(Y_batches) - 1):

    print(np.arange(splits[i], len(Y))[:, np.newaxis].shape)
    
    print(I.shape)
    
    idxs = [i for i in zip(np.arange(splits[i], len(Y)), I[:, 0])]
    print(idxs)
    
    print(index[np.arange(splits[i], len(Y))[:, np.newaxis], I])

(2, 1)
(2, 2)
[(np.int64(0), np.int32(11)), (np.int64(1), np.int32(1))]
[[11  1]
 [21 28]]


In [126]:
splits

array([0])

In [127]:
if i == 0 and i != (len(Y_batches) - 1):
    print(np.arange(0, splits[1])[:, np.newaxis].shape)
    print(I.shape)

    idxs = [i for i in zip(np.arange(0, splits[1]), I[:, 0])]
    print(idxs)
    
    print(index[np.arange(0, splits[1])[:, np.newaxis], I])

In [128]:
gamma.shape

(2, 2)

In [129]:
if i == (len(Y_batches) - 1):
    X[np.arange(splits[i], len(Y))[:, np.newaxis], I] = gamma
elif i == 0:
    X[np.arange(0, splits[1])[:, np.newaxis], I] = gamma

In [130]:
np.nonzero(X)

(array([0, 0, 1, 1]), array([ 1, 11,  1,  8]))

In [131]:
X, y = X_orig, y_orig

In [132]:
y.T.shape

(2, 50)

In [133]:
b_scipy2 = OMP_3(y.T, 3, X.T, debug=True)

Batch 0, Iteration 0
r shape: (2, 50)
[[7.6339965 ]
 [6.33859759]]
[1. 1.]
[7.6339965  6.33859759]
(2, 1)
D_I shape: (2, 1, 50)
y shape: (2, 50)
est shape: (2, 50)
(2, 50)
[2449425.07124399 1817892.16995816]
Batch 0, Iteration 1
r shape: (2, 50)
(2, 1, 50)
(2, 50, 1)
(2, 50, 2)
(2, 50)
original gamma shape: (2, 2, 1)
(2, 2)
D_I shape: (2, 2, 50)
y shape: (2, 50)
est shape: (2, 50)
(2, 50)
[1675390.91540181 1455231.95400867]
Batch 0, Iteration 2
r shape: (2, 50)
(2, 2, 50)
(2, 50, 1)
(2, 50, 3)
(2, 50)
original gamma shape: (2, 3, 1)
(2, 3)
D_I shape: (2, 3, 50)
y shape: (2, 50)
est shape: (2, 50)
(2, 50)
[1347560.04883038 1157900.67499184]
gamma: [[ 98.72352887 132.14938371  70.15323381]
 [136.86874232  75.95811465  67.35538119]]
gamma shape: (2, 3)
k shape: (2,)
I shape: (2, 3)



In [134]:
OMP_verif(b_scipy2)

[[  0.         132.14938371   0.           0.           0.
    0.           0.           0.          70.15323381   0.
    0.          98.72352887   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         136.86874232   0.           0.           0.
    0.           0.           0.          75.95811465   0.
    0.           0.           0.           0.           0.
    0.          67.35538119   0.           0.           0.        ]]
(array([0, 0, 0, 1, 1, 1]), array([ 1,  8, 11,  1,  8, 16]))
[132.14938371  70.15323381  98.72352887 136.86874232  75.95811465
  67.35538119]
